In [1]:
import requests
import os
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO

# Get all links from Catalog page:

In [2]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}

In [ ]:
# Define the URL you want to scrape
url = 'https://www.gucci.com/it/it/ca/women/ready-to-wear-for-women-c-women-readytowear'

In [ ]:
# Send an HTTP request to the URL
response = requests.get(url, headers = header)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
item_list = soup.select('article a')

In [ ]:
for item in item_list: 
    link = 'https://www.gucci.com' + item['href']
    print(link)

# Get from specific product page:
- Relevant pictures
- item description

In [6]:
# New Link
link = 'https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/skirts-for-women/midi-long-skirts-for-women/gg-supreme-oxford-cotton-skirt-p-788904ZAM9M5307'

In [7]:
# Send an HTTP request to the URL
response_1 = requests.get(link, headers = header)
soup_1 = BeautifulSoup(response_1.content, 'html.parser')

In [8]:
list = soup_1.find_all('picture' )


In [9]:
list = soup_1.find_all(attrs={"data-image-size": "small-retina"})

In [10]:
keeping = ['001_100','004_100','005_100']
# 001: Prod
# 004: back
# 005: 3/4 image


In [11]:
#keeping = ['001', '002', '004', '005', '012']

def contains_required_number(line):
    return any(num in line for num in keeping)

In [12]:
output_dir = "Img"
os.makedirs(output_dir, exist_ok=True)

In [13]:
i = 0
for item in list:
    i+=1 
    if (i == len(list)) | (i == 1):
        continue
    elif not(contains_required_number(item['srcset'])) :
        continue
    else:
        print(item['srcset'] ) 
        url = "https:" + item['srcset']

        # Download the image
        response = requests.get(url, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))
        # Save the image
        img_name = os.path.join(output_dir, f"Gucci5_{i}.jpg") 
        img_resized.save(img_name)
        print(f"Saved {img_name}")

//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805269/788904_ZAM9M_5307_001_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_2.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805271/788904_ZAM9M_5307_004_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_5.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805272/788904_ZAM9M_5307_005_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_6.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805274/788904_ZAM9M_5307_012_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_8.jpg
